In [6]:
import os
import rasterio
from shapely.geometry import box
import geopandas as gpd
import pandas as pd
from fiona.crs import from_epsg
import pycrs
from matplotlib import pyplot

In [7]:
os.chdir('C://Users//Scott/Desktop/exampleData/')
os.getcwd()

'C:\\Users\\Scott\\Desktop\\exampleData'

In [8]:
fp = r"MD_510-N-PAYSON-ST_2014.tif"
loadedGeotif = rasterio.open(fp)

In [10]:
epsg_code = int(loadedGeotif.crs.data['init'][5:])
print(epsg_code)

4326


In [13]:
loadedGeotif.bounds

BoundingBox(left=-76.64943837581602, bottom=39.29388854781503, right=-76.64903936757446, top=39.29419485412876)

In [17]:
northlat = loadedGeotif.bounds[3]
southlat = loadedGeotif.bounds[1]
westLong = loadedGeotif.bounds[0]
eastLong = loadedGeotif.bounds[2]

In [ ]:
# shrink csv

In [15]:
bmoreRealProperty = pd.read_csv('C://Users//Scott/Desktop/OpenBaltimoreData/baltimore_Real_Property.csv')  #load vacant homes cvs into pandas
#eventually link to ftp on baltimore open data website

C:\Users\Scott\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (8,28,68) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [21]:
list(bmoreRealProperty)

['OBJECTID',
 'PIN',
 'PINRELATE',
 'BLOCKLOT',
 'BLOCK',
 'LOT',
 'WARD',
 'SECTION',
 'ASSESSOR',
 'TAXBASE',
 'BFCVLAND',
 'BFCVIMPR',
 'LANDEXMP',
 'IMPREXMP',
 'CITYCRED',
 'STATCRED',
 'CCREDAMT',
 'SCREDAMT',
 'PERMHOME',
 'ASSESGRP',
 'LOT_SIZE',
 'NO_IMPRV',
 'CURRLAND',
 'CURRIMPR',
 'EXMPLAND',
 'EXMPIMPR',
 'FULLCASH',
 'EXMPTYPE',
 'EXMPCODE',
 'USEGROUP',
 'ZONECODE',
 'SDATCODE',
 'ARTAXBAS',
 'DISTSWCH',
 'DIST_ID',
 'STATETAX',
 'CITY_TAX',
 'AR_OWNER',
 'DEEDBOOK',
 'DEEDPAGE',
 'SALEDATE',
 'OWNER_ABBR',
 'OWNER_1',
 'OWNER_2',
 'OWNER_3',
 'FULLADDR',
 'STDIRPRE',
 'ST_NAME',
 'ST_TYPE',
 'BLDG_NO',
 'FRACTION',
 'UNIT_NUM',
 'SPAN_NUM',
 'SPANFRAC',
 'ZIP_CODE',
 'EXTD_ZIP',
 'DHCDUSE1',
 'DHCDUSE2',
 'DHCDUSE3',
 'DHCDUSE4',
 'DWELUNIT',
 'EFF_UNIT',
 'ROOMUNIT',
 'RPDELTAG',
 'RESPAGCY',
 'SALEPRIC',
 'PROPDESC',
 'NEIGHBOR',
 'SRVCCNTR',
 'YEAR_BUILD',
 'STRUCTAREA',
 'LDATE',
 'OWNMDE',
 'GRNDRENT',
 'SUBTYPE_GEODB',
 'SDATLINK',
 'BLOCKPLAT',
 'MAILTOADD',
 'V

In [38]:
#drop/keep columns
# Select the ones you want
bmoreRealProperty1 = bmoreRealProperty[['FULLADDR','OWNER_1','ZIP_CODE','VACIND', 'YEAR_BUILD', 'SALEDATE', 'SALEPRIC']]
bmoreRealProperty1[1:16]

,FULLADDR,OWNER_1,ZIP_CODE,VACIND,YEAR_BUILD,SALEDATE,SALEPRIC
1,2043 W NORTH AVE,"CHONG, YON UN",21217,NaN,0,11282005,5000
2,2041 W NORTH AVE,NEW YORK INC,21217,Y,1915,10171975,0
3,2039 W NORTH AVE,"MUHAMMAD, ANNA AVON",21217,Y,1920,11132014,0
4,2037 W NORTH AVE,"PRICE, WALTER (LIFE)",21217,N,1920,6202018,21000
5,2035 W NORTH AVE,NB5 BUSINESS TRUST,21217,Y,1915,1032008,0
6,2033 W NORTH AVE,"NORTH AVENUE REVELATION PROJECT,",21217,Y,1915,6072006,2120
7,2031 W NORTH AVE,"ALLTOGETHER COMMUNITY OUTREACH,",21217,Y,1920,4132015,2328
8,2029 W NORTH AVE,"DOUGLAS, VERONA E",21217,NaN,1915,3081995,10000
9,2027 W NORTH AVE,"SUMMERS, DAJUAN M",21217,NaN,1915,6302015,54000
10,2025 W NORTH AVE,"WHYTE, KAYVONNE",21217,NaN,1900,3212007,63000


In [29]:
len(bmoreRealProperty1)

237345

In [ ]:
nearbydf = bmoreRealProperty[(bmoreRealProperty['Latitude'] > 39.289053) & 
                            (bmoreRealProperty['Latitude'] < 39.2997794) &
                            (bmoreRealProperty['Longitude'] < -76.6408601)&
                            (bmoreRealProperty['Longitude'] > -76.661873)]

In [16]:
vacantBmore = pd.read_csv('C://Users//Scott/Desktop/OpenBaltimoreData/Vacant_Buildings_lat_long.csv')  #load vacant homes cvs into pandas
#eventually link to ftp on baltimore open data website

In [19]:
nearbydf = vacantBmore[(vacantBmore['lat'] > southlat) & 
                            (vacantBmore['lat'] < northlat) &
                            (vacantBmore['lon'] < eastLong)&
                            (vacantBmore['lon'] > westLong)]

In [20]:
nearbydf

,ReferenceID,Block,Lot,BuildingAddress,NoticeDate,Neighborhood,PoliceDistrict,CouncilDistrict,lat,lon
874,0118 063 031517,118,63,514 N PAYSON ST,3/15/2017,Midtown-Edmondson,Western,9,39.294116,-76.649242
875,0118 064 050919,118,64,512 N PAYSON ST,5/9/2019,Midtown-Edmondson,Western,9,39.294079,-76.649240
879,0118 065 072918,118,65,510 N PAYSON ST,7/29/2018,Midtown-Edmondson,Western,9,39.294041,-76.649238
880,0118 069 121409,118,69,502 N PAYSON ST,12/14/2009,MIDTOWN-EDMONDSON,WESTERN,9,39.293891,-76.649230


In [ ]:
def getNeighbors(geoTiff, bigdf, newdf): #geotiff, dataframe to parse, new dataframeName
    """Function to show image of vacant property"""
    
    loadedGeotif = rasterio.open(geoTiff)
    loadedGeotif.bounds
    #bounding box, assumes box is north south
    northlat = loadedGeotif.bounds[3]
    southlat = loadedGeotif.bounds[1]
    westLong = loadedGeotif.bounds[0]
    eastLong = loadedGeotif.bounds[2]
    
    nearbydf = vacantBmore[(vacantBmore['lat'] > southlat) & 
                            (vacantBmore['lat'] < northlat) &
                            (vacantBmore['lon'] < eastLong)&
                            (vacantBmore['lon'] > westLong)]
    #return df that has the addresses in the photo

In [39]:
# Set filepath (fix path relative to yours)
fp = "C://Users//Scott/Desktop/OpenBaltimoreData/Real_Property/Real_Property.shp"

# Read file using gpd.read_file()
realPropShape = gpd.read_file(fp)
type(realPropShape)

geopandas.geodataframe.GeoDataFrame

In [40]:
realPropShape.head()

,OBJECTID,PIN,PINRELATE,BLOCKLOT,BLOCK,LOT,WARD,SECTION,ASSESSOR,TAXBASE,...,OWNMDE,GRNDRENT,SUBTYPE_GE,SDATLINK,BLOCKPLAT,MAILTOADD,VACIND,ShapeSTAre,ShapeSTLen,geometry
0,1,0001001,0001001,0001 001,0001,001,15,370,329,141800,...,F,0.0,1,http://sdat.dat.maryland.gov/realproperty/page...,https://gis.baltimorecity.gov/zoning/blockplat...,"2045 W NORTH AVE, 21217",None,1359.932495,200.115609,"POLYGON ((1411050.735 598509.295, 1411050.905 ..."
1,2,0001002,0001002,0001 002,0001,002,15,370,329,49500,...,F,0.0,1,http://sdat.dat.maryland.gov/realproperty/page...,https://gis.baltimorecity.gov/zoning/blockplat...,"2045 W NORTH AVE, 21217",None,1174.104431,195.673505,"POLYGON ((1411050.735 598509.295, 1411064.715 ..."
2,3,0001003,0001003,0001 003,0001,003,15,370,308,7000,...,None,0.0,1,http://sdat.dat.maryland.gov/realproperty/page...,https://gis.baltimorecity.gov/zoning/blockplat...,"SUITE 201, 21208",Y,1188.327026,196.031880,"POLYGON ((1411083.735 598426.905, 1411069.505 ..."
3,4,0001004,0001004,0001 004,0001,004,15,370,308,7000,...,L,120.0,1,http://sdat.dat.maryland.gov/realproperty/page...,https://gis.baltimorecity.gov/zoning/blockplat...,"9726 MENDOZA RD RANDALLSTOWN, MD, 21133",Y,1201.343567,196.338977,"POLYGON ((1411098.045 598427.565, 1411083.735 ..."
4,5,0001005,0001005,0001 005,0001,005,15,370,308,21667,...,F,0.0,1,http://sdat.dat.maryland.gov/realproperty/page...,https://gis.baltimorecity.gov/zoning/blockplat...,"3112 MILFORD AVE, 21207",N,1106.556641,194.086967,"POLYGON ((1411111.235 598428.185, 1411098.045 ..."


In [24]:
list(realPropShape)

['OBJECTID',
 'PIN',
 'PINRELATE',
 'BLOCKLOT',
 'BLOCK',
 'LOT',
 'WARD',
 'SECTION',
 'ASSESSOR',
 'TAXBASE',
 'BFCVLAND',
 'BFCVIMPR',
 'LANDEXMP',
 'IMPREXMP',
 'CITYCRED',
 'STATCRED',
 'CCREDAMT',
 'SCREDAMT',
 'PERMHOME',
 'ASSESGRP',
 'LOT_SIZE',
 'NO_IMPRV',
 'CURRLAND',
 'CURRIMPR',
 'EXMPLAND',
 'EXMPIMPR',
 'FULLCASH',
 'EXMPTYPE',
 'EXMPCODE',
 'USEGROUP',
 'ZONECODE',
 'SDATCODE',
 'ARTAXBAS',
 'DISTSWCH',
 'DIST_ID',
 'STATETAX',
 'CITY_TAX',
 'AR_OWNER',
 'DEEDBOOK',
 'DEEDPAGE',
 'SALEDATE',
 'OWNER_ABBR',
 'OWNER_1',
 'OWNER_2',
 'OWNER_3',
 'FULLADDR',
 'STDIRPRE',
 'ST_NAME',
 'ST_TYPE',
 'BLDG_NO',
 'FRACTION',
 'UNIT_NUM',
 'SPAN_NUM',
 'SPANFRAC',
 'ZIP_CODE',
 'EXTD_ZIP',
 'DHCDUSE1',
 'DHCDUSE2',
 'DHCDUSE3',
 'DHCDUSE4',
 'DWELUNIT',
 'EFF_UNIT',
 'ROOMUNIT',
 'RPDELTAG',
 'RESPAGCY',
 'SALEPRIC',
 'PROPDESC',
 'NEIGHBOR',
 'SRVCCNTR',
 'YEAR_BUILD',
 'STRUCTAREA',
 'LDATE',
 'OWNMDE',
 'GRNDRENT',
 'SUBTYPE_GE',
 'SDATLINK',
 'BLOCKPLAT',
 'MAILTOADD',
 'VACI

In [42]:
realPropShape.iloc[1]

OBJECTID                                                      2
PIN                                                     0001002
PINRELATE                                               0001002
BLOCKLOT                                               0001 002
BLOCK                                                      0001
LOT                                                         002
WARD                                                         15
SECTION                                                     370
ASSESSOR                                                    329
TAXBASE                                                   49500
BFCVLAND                                                   7000
BFCVIMPR                                                  42500
LANDEXMP                                                      0
IMPREXMP                                                      0
CITYCRED                                                      0
STATCRED                                

In [48]:
# get centroid of geometry
realPropShape.centroid.y
# turn centroid into lat and long
# use lat and long to get neighbors in photo

0         598466.720763
1         598467.768920
2         598468.355367
3         598469.087418
4         598469.721015
5         598470.380763
6         598471.043163
7         598471.677145
8         598472.284277
9         598472.954359
10        598473.606296
11        598474.254873
12        598474.912872
13        598475.558882
14        598476.937479
15        598479.356417
16        598421.584805
17        598407.562467
18        598393.782718
19        598379.990407
20        598366.105355
21        598352.352475
22        598338.557106
23        598324.752595
24        598310.805321
25        598296.700002
26        598282.932091
27        598269.242491
28        598254.530916
29        598286.203612
              ...      
237315    584111.896159
237316    595586.341775
237317    600834.327700
237318    597228.943620
237319    595149.207643
237320    598343.532415
237321    605388.765978
237322    595988.799182
237323    586790.381357
237324    587443.252702
237325    591976

In [49]:
realPropShape.crs

{'proj': 'lcc',
 'lat_0': 37.6666666666667,
 'lon_0': -77,
 'lat_1': 38.3,
 'lat_2': 39.45,
 'x_0': 400000,
 'y_0': 0,
 'datum': 'NAD83',
 'units': 'us-ft',
 'no_defs': True}

In [51]:
realPropShape = realPropShape.to_crs(epsg=4326)

In [52]:
realPropShape.crs

{'init': 'epsg:4326', 'no_defs': True}

In [53]:
realPropShape.centroid.y

0         39.309413
1         39.309416
2         39.309418
3         39.309420
4         39.309421
5         39.309423
6         39.309424
7         39.309426
8         39.309428
9         39.309429
10        39.309431
11        39.309433
12        39.309434
13        39.309436
14        39.309439
15        39.309445
16        39.309287
17        39.309248
18        39.309210
19        39.309172
20        39.309134
21        39.309096
22        39.309059
23        39.309021
24        39.308982
25        39.308944
26        39.308906
27        39.308868
28        39.308828
29        39.308915
            ...    
237315    39.269687
237316    39.301214
237317    39.315570
237318    39.305724
237319    39.300014
237320    39.308775
237321    39.328016
237322    39.302356
237323    39.277421
237324    39.278846
237325    39.291294
237326    39.366438
237327    39.268253
237328    39.298496
237329    39.274139
237330    39.276955
237331    39.273191
237332    39.281618
237333    39.272364
